In [1]:
import warnings
from config import KAGGLE_DATA_PATH, KAGGLE_TARGET

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

Train: 3539
Test: 885


In [4]:
df = pd.read_csv(KAGGLE_DATA_PATH)
X = df.drop(KAGGLE_TARGET, axis=1)
y = df[KAGGLE_TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("Train:", X_train.shape[0])
print("Test:", X_test.shape[0])

Train: 3539
Test: 885


In [7]:
def remove_outliers(X_train, y_train):
    from sklearn.ensemble import IsolationForest

    isf = IsolationForest(n_jobs=-1, random_state=1)
    isf.fit(X_train)

    predictions = isf.predict(X_train)

    # The predict method returns 1 for inliers and -1 for outliers.
    X_train_filtered = X_train[predictions == 1]
    y_train_filtered = y_train[predictions == 1]
    return X_train_filtered, y_train_filtered


X_train, y_train = remove_outliers(X_train, y_train)
print("Train after removing outliers:", X_train.shape[0])
print("Test:", X_test.shape[0])

Train after removing outliers: 2553
Test: 885


In [8]:
from sklearn.ensemble import ExtraTreesClassifier

# Initialize the model
etc = ExtraTreesClassifier(n_estimators=100, random_state=1)
etc.fit(X_train, y_train)

# Get feature importances
feature_importances = etc.feature_importances_

# Sort the feature importances in descending order and get their indices
sorted_indices = np.argsort(feature_importances)[::-1]

# Print the features in descending order of importance
print("Feature ranking:")
for i in range(X_train.shape[1]):
    print(
        f"{i+1}. {X_train.columns[sorted_indices[i]]} ({feature_importances[sorted_indices[i]]})"
    )

# If you want to keep only top N features
N = 10  # For instance, keeping only top 10 features. Adjust this based on your requirements.
selected_features = X_train.columns[sorted_indices[:N]]

# Filter your data based on selected features
X_train_selected = X_train[selected_features]

Feature ranking:
1. Curricular units 2nd sem (approved) (0.13329039149197358)
2. Curricular units 1st sem (approved) (0.10609075781343576)
3. Curricular units 2nd sem (grade) (0.09111503677206818)
4. Curricular units 1st sem (grade) (0.06148314755513296)
5. Curricular units 2nd sem (evaluations) (0.04825560695272764)
6. Curricular units 1st sem (evaluations) (0.04344251449087888)
7. Father's occupation (0.033925363915045144)
8. Age at enrollment (0.03347628750561022)
9. Course (0.03322761532931514)
10. Mother's occupation (0.03197060339855032)
11. Mother's qualification (0.028797978545620724)
12. Unemployment rate (0.028230994935351812)
13. Father's qualification (0.028031030392488134)
14. Inflation rate (0.027184176843838358)
15. Application mode (0.027135604184733507)
16. GDP (0.026700235716730452)
17. Scholarship holder (0.025732218656565845)
18. Application order (0.02458162207885884)
19. Tuition fees up to date (0.023287554884522787)
20. Curricular units 2nd sem (enrolled) (0.0220